In [3]:
from collections import deque

# Grid
GRID_SIZE = 6


grid_letters = [
    ['A', 'A', 'A', 'B', 'B', 'C'],  # Row 1 (index 0)
    ['A', 'A', 'A', 'B', 'B', 'C'],  # Row 2 (index 1)
    ['A', 'A', 'B', 'B', 'C', 'C'],  # Row 3 (index 2)
    ['A', 'A', 'B', 'B', 'C', 'C'],  # Row 4 (index 3)
    ['A', 'B', 'B', 'C', 'C', 'C'],  # Row 5 (index 4)
    ['A', 'B', 'B', 'C', 'C', 'C'],  # Row 6 (index 5)
]

def position_to_coordinates(pos):
    col_mapping = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5}
    col = col_mapping[pos[0]]
    row = int(pos[1]) - 1
    return (col, row)

def coordinates_to_position(coord):
    col_mapping = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f'}
    col = col_mapping[coord[0]]
    row = str(coord[1] + 1)
    return col + row

def get_knight_moves(x, y):
    moves = [
        (x + 1, y + 2),
        (x + 1, y - 2),
        (x - 1, y + 2),
        (x - 1, y - 2),
        (x + 2, y + 1),
        (x + 2, y - 1),
        (x - 2, y + 1),
        (x - 2, y - 1),
    ]
    valid_moves = [
        (mx, my) for mx, my in moves
        if 0 <= mx < GRID_SIZE and 0 <= my < GRID_SIZE
    ]
    return valid_moves

def generate_all_paths(start, goal, max_depth=6):
    from collections import deque

    all_paths = []
    queue = deque()
    queue.append((start, [start]))
    while queue:
        current_pos, path = queue.popleft()
        if current_pos == goal:
            all_paths.append(path)
            continue
        if len(path) > max_depth:
            continue
        for neighbor in get_knight_moves(*current_pos):
            if neighbor not in path:
                queue.append((neighbor, path + [neighbor]))
    return all_paths

def generate_score_expression(path_coords):
    expression = None
    letters = []
    for i, coord in enumerate(path_coords):
        letter = grid_letters[coord[1]][coord[0]]
        letters.append(letter)

    current_letter = letters[0]
    expression = current_letter
    for i in range(1, len(letters)):
        next_letter = letters[i]
        if current_letter != next_letter:
            expression = f"({expression} * {next_letter})"
        else:
            expression = f"({expression} + {next_letter})"
        current_letter = next_letter
    return expression, letters

def find_common_abc_for_two_paths(start_pos1, goal_pos1, start_pos2, goal_pos2):
    start1 = position_to_coordinates(start_pos1)
    goal1 = position_to_coordinates(goal_pos1)
    start2 = position_to_coordinates(start_pos2)
    goal2 = position_to_coordinates(goal_pos2)


    all_paths1 = generate_all_paths(start1, goal1, max_depth=6)
    all_paths2 = generate_all_paths(start2, goal2, max_depth=6)


    expressions1 = []
    for path_coords in all_paths1:
        expression, letters = generate_score_expression(path_coords)
        expressions1.append((expression, letters, path_coords))

    expressions2 = []
    for path_coords in all_paths2:
        expression, letters = generate_score_expression(path_coords)
        expressions2.append((expression, letters, path_coords))

    print(f"Generated {len(expressions1)} paths from {start_pos1} to {goal_pos1}.")
    print(f"Generated {len(expressions2)} paths from {start_pos2} to {goal_pos2}.\n")


    max_value = 49  # Since A + B + C < 50
    for A_val in range(1, max_value):
        for B_val in range(1, max_value - A_val):
            for C_val in range(1, max_value - A_val - B_val):
                if A_val + B_val + C_val >= 50:
                    continue
                letter_values = {'A': A_val, 'B': B_val, 'C': C_val}

                valid_paths1 = []
                for expression, letters, path_coords in expressions1:
                    try:
                        expr_value = eval(expression, letter_values)
                        if expr_value == 2024:
                            valid_paths1.append((path_coords, letters))
                    except (OverflowError, ZeroDivisionError):
                        continue

                valid_paths2 = []
                for expression, letters, path_coords in expressions2:
                    try:
                        expr_value = eval(expression, letter_values)
                        if expr_value == 2024:
                            valid_paths2.append((path_coords, letters))
                    except (OverflowError, ZeroDivisionError):
                        continue
                # Check if there are valid paths for both pairs with the same A, B, C
                if valid_paths1 and valid_paths2:
                    print(f"Valid values found: A={A_val}, B={B_val}, C={C_val}")
                    print(f"Values sum to {A_val + B_val + C_val} (<50)")
                    print(f"\nPaths from {start_pos1} to {goal_pos1}:")
                    for path_coords, letters in valid_paths1:
                        path_positions = [coordinates_to_position(coord) for coord in path_coords]
                        print(f"Path: {' -> '.join(path_positions)}")
                        print(f"Letters: {' -> '.join(letters)}")
                    print(f"\nPaths from {start_pos2} to {goal_pos2}:")
                    for path_coords, letters in valid_paths2:
                        path_positions = [coordinates_to_position(coord) for coord in path_coords]
                        print(f"Path: {' -> '.join(path_positions)}")
                        print(f"Letters: {' -> '.join(letters)}")
                    print("-" * 40)
                    return  # Stop after finding the first valid solution
            print(f"Checked A={A_val}")
    print("No common values of A, B, C found that satisfy both paths.")

if __name__ == "__main__":
    start_position1 = 'a1'
    goal_position1 = 'f6'
    start_position2 = 'a6'
    goal_position2 = 'f1'
    find_common_abc_for_two_paths(start_position1, goal_position1, start_position2, goal_position2)


Generated 94 paths from a1 to f6.
Generated 94 paths from a6 to f1.

Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=1
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Checked A=2
Valid values found: A=2, B=13, C=22
Values sum to 37 (<50)

Paths from a1 to f6:
Path: a1 -> b3 -> c1 -> d3 -> c5 -> e4 -> f6
Letters: A -> A -> A -> B -> B -> C -> C
Path: a1 -> b3 -> c1 -> e2 -> c3 -> d5 -> f6
Letters: A 